# Training, hyperparameter tune, and deploy with TensorFlow

## Introduction
This tutorial shows how to train a simple deep neural network using the MNIST dataset and TensorFlow on Azure Machine Learning. MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of `28x28` pixels, representing number from 0 to 9. The goal is to create a multi-class classifier to identify the digit each image represents, and deploy it as a web service in Azure.

For more information about the MNIST dataset, please visit [Yan LeCun's website](http://yann.lecun.com/exdb/mnist/).



<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

Let's get started. First let's import some Python libraries.

In [ ]:
import sys
sys.version

In [ ]:
import datetime
now = datetime.datetime.now()
print(now)

In [ ]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML Version :", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

## Expérimentation

In [ ]:
from azureml.core import Experiment

script_folder = './sample_projects/workshop10'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='Exemple10-hyperparametertuning-tf')

## Données

In [ ]:
import urllib

os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')

In [ ]:
#pip install utils

In [ ]:
from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster.
X_train = load_data('./data/mnist/train-images.gz', False) / 255.0
y_train = load_data('./data/mnist/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/mnist/test-images.gz', False) / 255.0
y_test = load_data('./data/mnist/test-labels.gz', True).reshape(-1)

count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
    plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

## Create a Dataset for Files
A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. Dataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [ ]:
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [ ]:
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)
# list the files referenced by dataset
dataset.to_path()

## Azure ML Compute

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_NC6` GPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [ ]:
#Liste des compute servers définis
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

In [ ]:
%%time
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "clustergpuNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           min_nodes=1,
                                                           max_nodes=10)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

In [ ]:
# Statut du compute server
compute_target.list_nodes()

## Copy the training files into the script folder
The TensorFlow training script is already created for you. You can simply copy it into the script folder, together with the utility library used to load compressed data file into numpy array.

In [ ]:
import shutil

# the training logic is in the tf_mnist.py file.
shutil.copy('./tf_mnist.py', script_folder)

# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('./utils.py', script_folder)

## Construct neural network in TensorFlow
In the training script `tf_mnist.py`, it creates a very simple DNN (deep neural network), with just 2 hidden layers. The input layer has 28 * 28 = 784 neurons, each representing a pixel in an image. The first hidden layer has 300 neurons, and the second hidden layer has 100 neurons. The output layer has 10 neurons, each representing a targeted label from 0 to 9.



In [ ]:
with open(os.path.join(script_folder, './tf_mnist.py'), 'r') as f:
    print(f.read())

## Estimateur TensorFlow

Next, we construct an `azureml.train.dnn.TensorFlow` estimator object, use the Batch AI cluster as compute target, and pass the mount-point of the datastore to the training code as a parameter.

The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed -- if additional pip or conda packages are required, their names can be passed in via the `pip_packages` and `conda_packages` arguments and they will be included in the resulting docker.

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

In [ ]:
tags = {"Framework" : "TensorFlow" , "BatchSize" : "50" , "H1" : "100" , "H2" : "50" , "LR" : "0.01"}

In [ ]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 100,
    '--second-layer-neurons': 50,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='tf_mnist.py',
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

## Submit job to run
Submit the estimator to an Azure ML experiment to kick off the execution.

In [ ]:
run = exp.submit(est, tags=tags)

### Monitor the Run <a class="anchor" id="monitor-run"></a>
As the Run is executed, it will go through the following stages:
1. Preparing: A docker image is created matching the Python environment specified by the TensorFlow estimator and it will be uploaded to the workspace's Azure Container Registry. This step will only happen once for each Python environment -- the container will then be cached for subsequent runs. Creating and uploading the image takes about **5 minutes**. While the job is preparing, logs are streamed to the run history and can be viewed to monitor the progress of the image creation.

2. Scaling: If the compute needs to be scaled up (i.e. the Batch AI cluster requires more nodes to execute the run than currently available), the cluster will attempt to scale up in order to make the required amount of nodes available. Scaling typically takes about **5 minutes**.

3. Running: All scripts in the script folder are uploaded to the compute target, data stores are mounted/copied and the `entry_script` is executed. While the job is running, stdout and the `./logs` folder are streamed to the run history and can be viewed to monitor the progress of the run.

4. Post-Processing: The `./outputs` folder of the run is copied over to the run history

There are multiple ways to check the progress of a running job. We can use a Jupyter notebook widget. 

> Note: The widget will automatically update ever 10-15 seconds, always showing you the most up-to-date information about the run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
# Progression du run
run.get_details()

In [ ]:
run

> Prévoir 10 min de temps de traitement

Capture d'écran d'un run en cours de traitement :

<img src="https://github.com/retkowsky/images/blob/master/gpurun.jpg?raw=true">

### The Run object <a class="anchor" id="run-object"></a>
The Run object provides the interface to the run history -- both to the job and to the control plane (this notebook), and both while the job is running and after it has completed. It provides a number of interesting features for instance:
* `run.get_details()`: Provides a rich set of properties of the run
* `run.get_metrics()`: Provides a dictionary with all the metrics that were reported for the Run
* `run.get_file_names()`: List all the files that were uploaded to the run history for this Run. This will include the `outputs` and `logs` folder, azureml-logs and other logs, as well as files that were explicitly uploaded to the run using `run.upload_file()`

Below are some examples -- please run through them and inspect their output. 

In [ ]:
run.get_details()

In [ ]:
run.get_metrics()

In [ ]:
run.get_file_names()

## Plot accuracy over epochs
Since we can retrieve the metrics from the run, we can easily make plots using `matplotlib` in the notebook. Then we can add the plotted image to the run using `run.log_image()`, so all information about the run is kept together.

In [ ]:
os.makedirs('./graphiques', exist_ok=True)
metrics = run.get_metrics()

plt.figure(figsize = (13,5))
plt.plot(metrics['validation_acc'], 'r-', lw=4, alpha=.6)
plt.plot(metrics['training_acc'], 'b--', alpha=0.5)
plt.legend(['Données complètes', 'Mini-batch'])
plt.xlabel('epochs', fontsize=14)
plt.ylabel('accuracy', fontsize=14)
plt.title('Graphique Accuracy Epochs', fontsize=16)
run.log_image(name='graphiqueTF.png', plot=plt)
plt.show()

## Download du modèle

In [ ]:
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

## Intelligent hyperparameter tuning avec Azure HyperDrive

> https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100, 125),
        '--first-layer-neurons': choice(10, 50, 100, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 100, 200),
        '--learning-rate': loguniform(-6, -1)
    }
)

In [ ]:
est = TensorFlow(source_directory=script_folder,
                 script_params={'--data-folder': dataset.as_named_input('mnist').as_mount()},
                 compute_target=compute_target,
                 entry_script='tf_mnist.py', 
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

Now we will define an early termnination policy. The `BanditPolicy` basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric. No Extra costs.

In [ ]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [ ]:
hypparmtuning = HyperDriveConfig(estimator=est, 
                       hyperparameter_sampling=ps, 
                       policy=policy, 
                       primary_metric_name='validation_acc', 
                       primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                       max_total_runs=8,
                       max_concurrent_runs=6)

In [ ]:
tags = {"Framework" : "TensorFlow" , "Hyperdrive" : "Oui"}

In [ ]:
hypparmtuning = exp.submit(config=hypparmtuning, tags=tags)

In [ ]:
RunDetails(hypparmtuning).show()

> Temps de traitement : 10 min environ

In [ ]:
# Progression du run
hypparmtuning.get_details()

### Nous pouvons visualiser les résultats depuis l'experimentation Azure ML :

<img src="https://github.com/retkowsky/images/blob/master/hyperparam1.jpg?raw=true">

In [ ]:
hypparmtuning.wait_for_completion(show_output=True)

## Enregistrement du meilleur modèle

In [ ]:
best_run = hypparmtuning.get_best_run_by_primary_metric()

In [ ]:
print(best_run.get_file_names())

In [ ]:
model = best_run.register_model(model_name='Exemple10-Modele-TensorFlow', 
                                model_path='outputs/model', 
                                tags={'Framework':'TensorFlow', 'Hyperdrive':'Oui' , 'GPU':'Oui'})

In [ ]:
from azureml.core.model import Model
models = Model.list(ws, name="Exemple10-Modele-TensorFlow")
for m in models:
    print(m.name, "(version =", m.version,") - Tags=",m.tags)

### Liste des modèles référencés dans le workspace Azure ML :

In [ ]:
from azureml.core.model import Model
models = Model.list(ws)
for m in models:
    print(m.name, "- version =", m.version)

## Suppression compute target

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

In [ ]:
#compute_target.delete()

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">